In [1]:

import sys
import os

# Agregar el directorio raiz al PYTHONPATH
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


Cargar los datos recolectados fusionados

In [2]:
import pandas as pd
from config.config_scraped import MERGED_SCRAPED_DATA_PATH
from config.config_transcribe import TRANSCRIBED_VIDEOS_PATH

In [3]:
df_merged = pd.read_csv(MERGED_SCRAPED_DATA_PATH)

In [4]:
print(f"filas: {df_merged.shape[0]}")
print(f"coulmnas: {df_merged.shape[1]}")

filas: 7108
coulmnas: 8


¿Cuantas urls repetidas hay?

In [5]:
print(df_merged["url"].unique().shape)
print(df_merged.drop_duplicates(subset="url", keep="first").reset_index(drop=True).shape)
df1 = df_merged.drop_duplicates(subset="url", keep="first").reset_index(drop=True)
print(df1.info())

(6398,)
(6398, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6398 entries, 0 to 6397
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   views               6398 non-null   int64  
 1   title               4107 non-null   object 
 2   hashtags            5838 non-null   object 
 3   date                6398 non-null   object 
 4   fecha_recuperacion  6398 non-null   object 
 5   caption             0 non-null      float64
 6   url                 6398 non-null   object 
 7   termino             6398 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 400.0+ KB
None


# Datos con fechas reconstruidas

Colocar el archivo con las fechas previamente reconstruidas en la ruta TRANSCRIBED_VIDEOS-PATH.

In [6]:
df_fechas_rec = pd.read_csv(TRANSCRIBED_VIDEOS_PATH)

In [7]:

print(f"filas: {df_fechas_rec.shape[0]}")
print(f"coulmnas: {df_fechas_rec.shape[1]}")
df2 = df_fechas_rec.drop_duplicates(subset="url", keep="first").reset_index(drop=True)
print(df2.info())

filas: 6059
coulmnas: 10
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6059 entries, 0 to 6058
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   fecha_publicacion  6059 non-null   object
 1   termino            6059 non-null   object
 2   vistas             6059 non-null   int64 
 3   url                6059 non-null   object
 4   titulo             3931 non-null   object
 5   hashtags           5511 non-null   object
 6   descargado         6059 non-null   bool  
 7   text               5753 non-null   object
 8   idioma             5966 non-null   object
 9   polaridad          5149 non-null   object
dtypes: bool(1), int64(1), object(8)
memory usage: 432.1+ KB
None


Hubo 7108 videos recolectados en total

De los cuales 6398 son videos únicos o urls únicas.

Y solamente se obtuvo la transcripción de 6059 videos de los 6398 videos únicos.

In [8]:
merged_df = df2.merge(df1, on='url', how='left')


In [9]:
print(merged_df.info())
print(merged_df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6059 entries, 0 to 6058
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fecha_publicacion   6059 non-null   object 
 1   termino_x           6059 non-null   object 
 2   vistas              6059 non-null   int64  
 3   url                 6059 non-null   object 
 4   titulo              3931 non-null   object 
 5   hashtags_x          5511 non-null   object 
 6   descargado          6059 non-null   bool   
 7   text                5753 non-null   object 
 8   idioma              5966 non-null   object 
 9   polaridad           5149 non-null   object 
 10  views               6059 non-null   int64  
 11  title               3931 non-null   object 
 12  hashtags_y          5511 non-null   object 
 13  date                6059 non-null   object 
 14  fecha_recuperacion  6059 non-null   object 
 15  caption             0 non-null      float64
 16  termin


Reconstruir la fecha de publicacion

In [10]:

from src.limpieza.limpieza import corregir_fecha_tiktok
merged_df['fecha_publicacion_y'] = merged_df.apply(
    lambda row: corregir_fecha_tiktok(row['date'], row['fecha_recuperacion']),
    axis=1
)

In [11]:
merged_df['fecha_publicacion'] = pd.to_datetime(merged_df['fecha_publicacion'])
merged_df['fecha_publicacion_y'] = pd.to_datetime(merged_df['fecha_publicacion_y'])
print(merged_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6059 entries, 0 to 6058
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   fecha_publicacion    6059 non-null   datetime64[ns]
 1   termino_x            6059 non-null   object        
 2   vistas               6059 non-null   int64         
 3   url                  6059 non-null   object        
 4   titulo               3931 non-null   object        
 5   hashtags_x           5511 non-null   object        
 6   descargado           6059 non-null   bool          
 7   text                 5753 non-null   object        
 8   idioma               5966 non-null   object        
 9   polaridad            5149 non-null   object        
 10  views                6059 non-null   int64         
 11  title                3931 non-null   object        
 12  hashtags_y           5511 non-null   object        
 13  date                 6059 non-nul

Verificamos que los hastags, terminos, etc sean los mismos

In [12]:
termino_igual = merged_df['termino_x'] == merged_df['termino_y']
hashtags_igual = merged_df['hashtags_x'] == merged_df['hashtags_y']
vistas_igual = merged_df['views'] == merged_df['vistas']
titulo_igual = merged_df['title'] == merged_df['titulo']
fecha_igual = merged_df['fecha_publicacion'] == merged_df['fecha_publicacion_y']

# Mostrar filas donde hay diferencias
diferencias_termino = merged_df[~termino_igual][['url', 'termino_x', 'termino_y']]
diferencias_hashtags = merged_df[~hashtags_igual][['url', 'hashtags_x', 'hashtags_y']]
diferencias_vistas = merged_df[~vistas_igual][['url', 'views', 'vistas']]
diferencias_titulo = merged_df[~titulo_igual][['url', 'title', 'titulo']]
diferencias_fecha = merged_df[~fecha_igual][['url', 'fecha_publicacion', 'fecha_publicacion_y']]

print("Diferencias en 'termino':")
print(diferencias_termino)

print("\nDiferencias en 'hashtags':")
print(diferencias_hashtags)

print("\nDiferencias en 'vistas':")
print(diferencias_vistas)

print("\nDiferencias en 'titulo':")
print(diferencias_titulo)

print("\nDiferencias en 'fecha':")
print(diferencias_fecha)

# print(termino_igual)
# print(hashtags_igual)

Diferencias en 'termino':
Empty DataFrame
Columns: [url, termino_x, termino_y]
Index: []

Diferencias en 'hashtags':
                                                    url hashtags_x hashtags_y
20    https://www.tiktok.com/@romanntiqa/video/74497...        NaN        NaN
22    https://www.tiktok.com/@claudiagarmaa/video/74...        NaN        NaN
24    https://www.tiktok.com/@unotv/video/7462445502...        NaN        NaN
37    https://www.tiktok.com/@nachopla12/video/72208...        NaN        NaN
38    https://www.tiktok.com/@luciallamasmartinez/vi...        NaN        NaN
...                                                 ...        ...        ...
5998  https://www.tiktok.com/@chicavainilla/video/72...        NaN        NaN
6022  https://www.tiktok.com/@sanando.ando/video/742...        NaN        NaN
6030  https://www.tiktok.com/@thetechbadger/video/73...        NaN        NaN
6046  https://www.tiktok.com/@isabelacastroa/video/7...        NaN        NaN
6051  https://www.tiktok.

In [13]:
# Verificar si las diferencias son solo NaN
diferencias_hashtags = merged_df['hashtags_x'] != merged_df['hashtags_y']
solo_nan = merged_df['hashtags_x'].isna() | merged_df['hashtags_y'].isna()

# Si todas las diferencias son por NaN, entonces no hay diferencias reales
if (diferencias_hashtags == solo_nan).all():
    print("No hay diferencias reales en 'hastags', solo valores NaN.")
else:
    print("Hay diferencias reales en 'hastags'.")

# Verificar si las diferencias son solo NaN
diferencias_titulo = merged_df['title'] != merged_df['titulo']
solo_nan = merged_df['title'].isna() | merged_df['titulo'].isna()

# Si todas las diferencias son por NaN, entonces no hay diferencias reales
if (diferencias_titulo == solo_nan).all():
    print("No hay diferencias reales en 'titulo', solo valores NaN.")
else:
    print("Hay diferencias reales en 'titulo'.")
    
    

No hay diferencias reales en 'hastags', solo valores NaN.
No hay diferencias reales en 'titulo', solo valores NaN.


Eliminar columnas duplicadas, después de comprobar que tienen los mismos valores

In [14]:
merged_df = merged_df.drop(columns=['hashtags_y']).rename(columns={'hashtags_x': 'hashtags'})
merged_df = merged_df.drop(columns=['termino_y']).rename(columns={'termino_x': 'termino'})
merged_df = merged_df.drop(columns=['title'])
merged_df = merged_df.drop(columns=['views'])
merged_df = merged_df.drop(columns=['fecha_publicacion_y'])


merged_df = merged_df.drop(columns=['caption']) # Este campo es eliminado al obtener la transcripción, es igual al titulo
merged_df = merged_df.drop(columns=['fecha_publicacion']) # Será reconstruida en el EDA


In [15]:
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6059 entries, 0 to 6058
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   termino             6059 non-null   object
 1   vistas              6059 non-null   int64 
 2   url                 6059 non-null   object
 3   titulo              3931 non-null   object
 4   hashtags            5511 non-null   object
 5   descargado          6059 non-null   bool  
 6   text                5753 non-null   object
 7   idioma              5966 non-null   object
 8   polaridad           5149 non-null   object
 9   date                6059 non-null   object
 10  fecha_recuperacion  6059 non-null   object
dtypes: bool(1), int64(1), object(9)
memory usage: 479.4+ KB
None



Reordenar columnas

In [16]:
cols = ["date", "fecha_recuperacion", "termino", "vistas", "url", "titulo", "hashtags", "descargado", "text", "idioma", "polaridad"]
merged_df = merged_df[cols] # Reordenar

⚠ Guardar uno con polaridad y otro sin polaridad para el eda

In [17]:

merged_df.to_csv("transcribed_polaridad.csv", index=False)

In [18]:
merged_df = merged_df.drop(columns=['polaridad'])
merged_df.to_csv("transcribed1.csv",index=False)